# Import packages and data 

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import scipy.stats
import anndata
import matplotlib.pyplot as plt
import matplotlib as mpl
import rpy2

In [2]:
sc.settings.verbosity = 1  # verbosity: errors (0), warnings (1), info (2), hints (3)
# Set up the plot config for viewing the annotation clearly.
sc.settings.set_figure_params(dpi=120, dpi_save=1000)
sc.logging.print_versions()

scanpy==1.4.4 anndata==0.7.1 umap==0.3.10 numpy==1.17.1 scipy==1.4.1 pandas==0.23.0 scikit-learn==0.22.2.post1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Save UMAP for Monocle3 from the processed dataset

In [4]:
adata = sc.read("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs_mk3/data/figs8e_fbm_eryth_dr_20210429.h5ad")

In [5]:
# save the info on variable genes used in scanpy so these can be transferred to monocle
gene_df = adata.var

In [6]:
pd.DataFrame(adata.obsm["X_draw_graph_fa"]).to_csv("/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs_mk3/resources_for_pipelines/fbm_eryth_fdg_20210429.csv")

# Load the fetal BM cells from the raw dataset

In [7]:
adata = sc.read('/Users/b8058304/Documents/PhD_work/Coding/bm_plus_19pcw/data/bm_plus_19pcw_raw_dr_20201007.h5ad')

In [8]:
adata.raw = adata

In [9]:
cell_numbers = adata.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
CD4 T cell                   327
CD8 T cell                   171
CD14 monocyte               8763
CD56 bright NK               449
CMP                          425
DC1                           50
DC2                          598
DC3                          705
DC precursor                 201
ELP                         1357
GMP                         1281
HSC                           92
ILC precursor                 67
LMPP                          34
MEMP                          16
MEP                          269
MK                          1000
MOP                         3838
MPP myeloid                   92
NK T cell                    111
NK progenitor                 26
Treg                          62
adipo-CAR                    353
arteriolar fibroblast         83
basophil                     139
chondrocyte                   80
early MK                    1624
early erythroid             7474
early osteoblast             280
endosteal fibroblast          5

In [10]:
celltypes = ['HSC', 'MEMP', 'MEP', 'early erythroid']

In [11]:
adata = adata[adata.obs['cell.labels'].isin(celltypes)].copy()

In [12]:
cell_numbers = adata.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
HSC                  92
MEMP                 16
MEP                 269
early erythroid    7474
dtype: int64

In [13]:
adata.obs['cell.labels'] = adata.obs['cell.labels'].replace(["MEP", "MEMP"], ["MEP_MEMP", "MEP_MEMP"])

In [14]:
cell_numbers = adata.obs.groupby(["cell.labels"]).apply(len)
cell_numbers

cell.labels
HSC                  92
MEP_MEMP            285
early erythroid    7474
dtype: int64

In [15]:
adata.var = gene_df

In [16]:
adata.shape

(7851, 33712)

# Convert to CDS

# Load rpy2 link

In [17]:
# convert anndata to sce
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

In [18]:
%R d = as.data.frame( R.Version() )
%R d = d['version.string']

,version.string
1,R version 3.6.2 (2019-12-12)


In [19]:
%%R 

library(scran)
library(RColorBrewer)
library(slingshot)
library(monocle)
library(gam)
library(clusterExperiment)
library(ggplot2)
library(plyr)

R[write to console]: Loading required package: SingleCellExperiment

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: GenomicRanges

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find

# Convert Scanpy object to CellDataSet object and prepare obj

In [20]:
%%R -i adata
adata

class: SingleCellExperiment 
dim: 33712 7851 
metadata(0):
assays(1): X
rownames(33712): RP11-34P13.3 RP11-34P13.7 ... CTB-58E17.2
  RP5-1077H22.1
rowData names(6): gene_ids.1 feature_types.1 ... dispersions
  dispersions_norm
colnames(7851):
  F21_boneMarrow_CD45+_FCAImmP7179367_3prime_AAACGGGGTAAGTAGT
  F21_boneMarrow_CD45+_FCAImmP7179367_3prime_CCGGTAGGTCCGTGAC ...
  TTGCATTGTGTGTGTT-1-WSSS_F_BON8710643
  TTTCAGTCAGGCCCTA-1-WSSS_F_BON8710643
colData names(23): cell.labels doublets ... nk_meta mito.threshold
reducedDimNames(3): X_orig_pca PCA UMAP
spikeNames(0):
altExpNames(0):


In [21]:
#Preprocessing for monocle
data_mat_mon = adata.X.T
var_mon=adata.var.copy()
obs_mon=adata.obs.copy()

In [22]:
var_mon

,gene_ids-1,feature_types-1,highly_variable,means,dispersions,dispersions_norm
RP11-34P13.3,ENSG00000243485,Gene Expression,False,7.672502e-04,0.674484,0.874231
RP11-34P13.7,ENSG00000238009,Gene Expression,False,2.521528e-03,-0.009908,-0.507574
RP11-34P13.8,ENSG00000239945,Gene Expression,False,3.579154e-04,1.033361,1.598814
RP11-34P13.9,ENSG00000241599,Gene Expression,False,9.418574e-05,-0.301798,-1.096907
FO538757.3,ENSG00000279928,Gene Expression,True,1.495981e-02,0.640760,0.806141
FO538757.2,ENSG00000279457,Gene Expression,False,7.443020e-01,0.617422,-0.018670
AP006222.2,ENSG00000228463,Gene Expression,False,1.779741e-01,0.243973,0.005020
RP4-669L17.10,ENSG00000237094,Gene Expression,False,1.237333e-02,0.180426,-0.123284
RP5-857K21.4,ENSG00000230021,Gene Expression,False,1.153280e-02,0.137920,-0.209105
RP11-206L10.9,ENSG00000237491,Gene Expression,True,5.594313e-02,0.490898,0.503567


In [23]:
%%R -i data_mat_mon -i obs_mon -i var_mon

#Set up the CellDataSet data structure
print("setting up cell dataset structure")
print("printing pd - cells")
pd <- AnnotatedDataFrame(data = obs_mon)
print(pd)
print("printing fd - genes")
fd <- AnnotatedDataFrame(data = var_mon)
print(fd)
# assigning cells as column name and genes as rownames of GEX matrix (data_mat_mon)
colnames(data_mat_mon) <- rownames(pd)
rownames(data_mat_mon) <- rownames(fd)
print("create newCellDataSet using the matrix, cells and genes")
ie_regions_cds <- newCellDataSet(cellData=data_mat_mon, phenoData=pd, featureData=fd, expressionFamily=negbinomial.size())

# print ie_regions_cds
print("printing ie_regions_cds")
print(ie_regions_cds)

[1] "setting up cell dataset structure"
[1] "printing pd - cells"
An object of class 'AnnotatedDataFrame'
  rowNames: F21_boneMarrow_CD45+_FCAImmP7179367_3prime_AAACGGGGTAAGTAGT
    F21_boneMarrow_CD45+_FCAImmP7179367_3prime_CCGGTAGGTCCGTGAC ...
    TTTCAGTCAGGCCCTA-1-WSSS_F_BON8710643 (7851 total)
  varLabels: cell.labels doublets ... mito.threshold (23 total)
  varMetadata: labelDescription
[1] "printing fd - genes"
An object of class 'AnnotatedDataFrame'
  rowNames: RP11-34P13.3 RP11-34P13.7 ... RP5-1077H22.1 (33712 total)
  varLabels: gene_ids.1 feature_types.1 ... dispersions_norm (6 total)
  varMetadata: labelDescription
[1] "create newCellDataSet using the matrix, cells and genes"
[1] "printing ie_regions_cds"
CellDataSet (storageMode: environment)
assayData: 33712 features, 7851 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames:
    F21_boneMarrow_CD45+_FCAImmP7179367_3prime_AAACGGGGTAAGTAGT
    F21_boneMarrow_CD45+_FCAImmP7179367_3prime_CCGGTAGGTCCGTG

# Within CDS, select HVGs (previously defined in scanpy)

In [24]:
%%R 

#Filter highly variable genes from our analysis (normalisation by log and size factor will be performed in monocle3)
print("filter hvgs")
hvg_mask = fData(ie_regions_cds)$highly_variable
ie_regions_cds <- ie_regions_cds[hvg_mask,]

# print ie_regions_cds
print("printing cds again")
print(ie_regions_cds)

[1] "filter hvgs"
[1] "printing cds again"
CellDataSet (storageMode: environment)
assayData: 3059 features, 7851 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames:
    F21_boneMarrow_CD45+_FCAImmP7179367_3prime_AAACGGGGTAAGTAGT
    F21_boneMarrow_CD45+_FCAImmP7179367_3prime_CCGGTAGGTCCGTGAC ...
    TTTCAGTCAGGCCCTA-1-WSSS_F_BON8710643 (7851 total)
  varLabels: cell.labels doublets ... Size_Factor (24 total)
  varMetadata: labelDescription
featureData
  featureNames: FO538757.3 RP11-206L10.9 ... MATR3-1 (3059 total)
  fvarLabels: gene_ids.1 feature_types.1 ... dispersions_norm (6 total)
  fvarMetadata: labelDescription
experimentData: use 'experimentData(object)'
Annotation:  


# Save object for input into Monocle3

In [25]:
%%R 

saveRDS(ie_regions_cds, 
        "/Users/b8058304/Documents/PhD_work/Coding/manuscript_figs_mk3/data/figs8e_fbm_eryth_cds_20210429.RDS")